## Import Data & Libraries

In [1]:
!rm -rf Bot-sp
!git clone --recursive https://github.com/luukru/Bot-sp.git
!mv Bot-sp Bot_sp # rename 
!cp -r Bot_sp/raw_data .

Cloning into 'Bot-sp'...
remote: Counting objects: 38, done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 38 (delta 13), reused 38 (delta 13), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [0]:
# Dit hoeft alleen als we Farizrahman's Seq2seq gebruiken

# !rm -rf seq2seq
# !rm -rf recurrentshop
# !rm -rf recurrentshop-repo
# !rm -rf recurrentshop.egg-info
# !rm -rf seq2seq.egg-info
# !rm -rf seq2seq-repo
# !rm -rf build 
# !rm -rf dist
# !ls

In [0]:
# Dit hoeft alleen als we Farizrahman's Seq2seq gebruiken

# !git clone --recursive https://github.com/luukru/recurrentshop.git
# !git clone --recursive https://github.com/luukru/seq2seq.git

In [0]:
# Dit hoeft alleen als we Farizrahman's Seq2seq gebruiken


# !mv recurrentshop recurrentshop-repo
# !mv seq2seq seq2seq-repo

In [0]:
# Dit hoeft alleen als we Farizrahman's Seq2seq gebruiken


# !ls
# !cp -r recurrentshop-repo/recurrentshop .
# !cp -r seq2seq-repo/seq2seq .
# !ls

In [0]:
!mkdir ckpt
#!ls 
!ls

Bot	 idx_a.npy     seq2seq_model.ckpt-3000.data-00000-of-00001
Bot_sp	 idx_q.npy     seq2seq_model.ckpt-3000.index
ckpt	 metadata.pkl  seq2seq_model.ckpt-3000.meta
datalab  raw_data


## Imports

In [2]:
import pandas as pd
import numpy as np

# import seq2seq
# from seq2seq.models import SimpleSeq2Seq

from Bot_sp import seq2seq_wrapper
from Bot_sp import data
from Bot_sp import data_utils
!pip install pydrive

    100% |████████████████████████████████| 993kB 1.1MB/s 
  Running setup.py bdist_wheel for pydrive ... - \ done
  Stored in directory: /content/.cache/pip/wheels/ea/bd/b1/14e948c56cce2c7ac1589346cb54d7efb659fa2ddfa6abba88
Successfully built pydrive


## Load Data

In [0]:
# Take the questions and answers from the Kaggle tv-series dataset.
def get_tv_series_q_and_a():
  # Read South Park data
  data = pd.read_csv('raw_data/All-seasons.csv')
  data = np.array(data)

  # Input & output for South park
  questions = []
  answers = []

  # Find Cartman's lines and corresponding input
  for i in range(0,len(data)):
      if data[i,2] == 'Cartman':
          questions.append(data[i-1,3])
          answers.append(data[i,3])
  
  return questions, answers

In [0]:
# Take the questions and answers from the Cornell Movie dataset.
def get_movie_q_and_a():
    # Get the raw movie lines and conversations
    line_data = open('raw_data/southpark_lines.txt').read().split('\n')
    conv_data = open('raw_data/southpark_conversations.txt').read().split('\n')
    
    # Put the movie lines into a dictionary, with key = line-ID and value = [character, text].
    lines = {}
    for line in line_data:
        line = line.split(' +++$+++ ')
        lines[line[0]] = [line[3], line[4]]
    
    # Get the lines for each conversation involving Cartman
    convs = []
    for conv in conv_data:
        conv = eval(conv.split(' +++$+++ ')[3])
        # If Cartman is in the conversation, store the whole conversation.
        if len([1 for line in conv if (lines[line][0] == 'CARTMAN')]) > 1:
            convs.append([lines[line] for line in conv])

    # Extract questions and answers
    questions = []
    answers = []
    for conv in convs:
        for i in range(0, len(conv)-1):
            if conv[i][0] != 'CARTMAN' and conv[i+1][0] == 'CARTMAN':
                questions.append(conv[i][1])
                answers.append(conv[i+1][1])

    return questions, answers

In [0]:
# Return the questions and answers from both the tv_series and the movie.
def get_combined_q_and_a():
  questions_tv, answers_tv = get_tv_series_q_and_a()
  questions_mv, answers_mv = get_movie_q_and_a()

  questions = questions_tv + questions_mv
  answers = answers_tv + answers_mv
  
  return questions, answers

## Data Statistics

In [0]:
# Display lines and words per main character in the show.
# Display questions and answers involving Cartman as answerer.
# Display a sample of 10 question and answer pairs.
def show_tv_series_statistics():
  data = pd.read_csv('raw_data/All-seasons.csv')
  data = np.array(data)

  for character in ['Cartman', 'Stan', 'Kyle', 'Kenny']:
      line_count = 0
      word_count = 0
      print("{}:".format(character))
      for row in data:
              if row[2] == character:
                  line_count += 1
                  word_count += len(row[3].split(' '))
      print("\tLine count: {}".format(line_count))
      print("\tWord_count: {}\n".format(word_count))

  questions, answers = get_tv_series_q_and_a()

  print("Statistics:")
  print('\tQuestions: {}'.format(len(questions)))
  print('\tOutputs: {}\n'.format(len(answers)))

  # Print sample of 'questions' plus Cartman's answers
  print("Sample:")
  for i in range(0,10):
      print('\tQuestion: %r\n\tCartman: %r' % (questions[i], answers[i]))

In [0]:
# Display questions and answers involving Cartman from the movie.
def show_movie_statistics():
  questions, answers = get_movie_q_and_a()
  
  print("Statistics:")
  print('\tQuestions: {}'.format(len(questions)))
  print('\tAnswers: {}\n'.format(len(answers)))

  # Print sample of 'questions' plus Cartman's answers
  print("Sample:")
  for i in range(0,10):
      print('\tQuestion: %r\n\tCartman: %r' % (questions[i], answers[i]))

In [0]:
show_tv_series_statistics()

Cartman:
	Line count: 9774
	Word_count: 133855

Stan:
	Line count: 7680
	Word_count: 70623

Kyle:
	Line count: 7099
	Word_count: 65243

Kenny:
	Line count: 881
	Word_count: 4413

Statistics:
	Questions: 9774
	Outputs: 9774

Sample:
	Question: "I hope you're making the right choice.\n"
	Cartman: "I'm gonna miss him.  I'm gonna miss Chef and I...and I don't know how to tell him! \n"
	Question: 'Draw two card, fatass.\n'
	Cartman: 'Reverse to you, Jew. \n'
	Question: 'Yeah!\n'
	Cartman: 'All right! \n'
	Question: 'Right. Uh, see ya.\n'
	Cartman: 'Uh, guys? Did Chef seem a little, uh, trippy to you?\n'
	Question: "It's really weird what he said. I don't know, it kind of confused me.\n"
	Cartman: "Oh boy oh boy, I can't wait to have Chef's lunch food again.\n"
	Question: 'Well, how about I meet you boys after work and we make love?\n'
	Cartman: 'Excuse me?\n'
	Question: 'I specializes in your asshole, Kyle. \n'
	Cartman: "...Man, I can't believe all this time, Chef just wanted us for sex.\n

In [0]:
show_movie_statistics()

## Preprocess Data

In [0]:
def filter_line(line):
  whitelist = ['0123456789abcdefghijklmnopqrstuvwxyz']
  return ''.join([ char for char in line if char in whitelist ])

In [0]:
def preprocess_data():
  questions, answers = get_combined_q_and_a()
  
  # Conver to lowercase
  questions = [ question.lower() for question in questions ] 
  answers = [ answer.lower() for answer in answers ]
  
  # Filter out non alphanumerics
  questions = [ filter_line(question) for question in questions ]
  answers = [ filter_line(answer) for answer in answers ]
  
  return questions, answers

In [0]:
# Replace unknown words with synonyms that are known
from nltk.corpus import wordnet as wn

def get_synonyms(word):
    synonyms = []
    for syn in wn.synsets(word):
        for l in syn.lemmas():
            synonyms.append(l.name())
    return synonyms

def synonyms(sentence, vocab):
    for word in sentence:
        if word not in vocab:
            synonyms = get_synonyms(word)
            if len(synonyms) > 0:
              for syn in synonyms:
                  if syn in vocab:
                      sentence[sentence.index(word)] = syn
    return sentence

## Seq2seq

In [9]:
# Preprocess data
questions, answers = get_combined_q_and_a()
data.process_data(questions, answers)


>> Filter lines

>> 2nd layer of filtering
30% filtered from original data
q : [this eight year old son of a single parent just wont behave]; a : [wir mssen die juden ausrotten ]
q : [its time for nanny stella to show eric cartman his ways are not going to be tolerated anymore ]; a : [mom are you deaf somebodys at the door]
q : [yes i think its the nanny boopsiekins]; a : [killer im gonna be on tv now ]
q : [and you must be eric]; a : [mom i want a twinkie]

>> Segment lines into words

:: Sample from segmented list of words
q : [['this', 'eight', 'year', 'old', 'son', 'of', 'a', 'single', 'parent', 'just', 'wont', 'behave']]; a : [['wir', 'mssen', 'die', 'juden', 'ausrotten']]
q : [['its', 'time', 'for', 'nanny', 'stella', 'to', 'show', 'eric', 'cartman', 'his', 'ways', 'are', 'not', 'going', 'to', 'be', 'tolerated', 'anymore']]; a : [['mom', 'are', 'you', 'deaf', 'somebodys', 'at', 'the', 'door']]
q : [['yes', 'i', 'think', 'its', 'the', 'nanny', 'boopsiekins']]; a : [['killer', 'im

In [0]:
# Split into train, test and validation set
metadata, idx_q, idx_a = data.load_data(PATH='')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [0]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = len(metadata['idx2w'])  
emb_dim = 1024

In [12]:
# Seq2seq model
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/',
                               emb_dim=emb_dim,
                               num_layers=3,
                                epochs=100000) # Hou dit hoog, want hij saved om de epochs/100 iterations en dat vertraagt de boel.

<log> Building Graph </log>

In [0]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 32)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

# TODO: Moeten we misschien deze batches vastleggen, zodat ze niet tussen training sesseis veranderen? (saven op drive ofzo?)
# Ik weet niet of t nu een probleem oplevert.

In [16]:
# NOTE: Als je de kernel interrupt blijft de laatst gesavede checkpoint geldig.


# Restore session to continue training
sess = model.restore_last_session()

# Continue training the session
sess = model.train(train_batch_gen, val_batch_gen, sess)

# Train new session (that overwrites old sessions unless we fix that)
# sess = model.train(train_batch_gen, val_batch_gen)

INFO:tensorflow:Restoring parameters from ckpt/seq2seq_model.ckpt-31000

<log> Training started </log>
Interrupted by user at iteration 206


In [17]:
sess = model.restore_last_session()
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

INFO:tensorflow:Restoring parameters from ckpt/seq2seq_model.ckpt-31000
(256, 25)


In [18]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

q : [well wait a minute uhhh boys do you think you can go get that tape back for us really fast]; a : [what i didnt]
q : [dont go anywhere russell crowe will be right back after a few more commercials]; a : [that will be anyone to the money in the rally of the doors is strictly fun policy]
q : [can i help you]; a : [stay is a window hee hee you just be me and iand i have to prove to watch you the time that thai is in]
q : [yes this shitty wok]; a : [i have a deal for the dwarf of in is a bad mall]
q : [scott tenorman is an eighth grader who sold cartman his pubes for ten dollars and now cartmans all pissed off]; a : [ohohh let mephesto wish is an words in you got the parents molestered us]
q : [no ike you cant come to school with me]; a : [yeah go my lies are a whore line i have a contract contest]
q : [im so glad were not murderers]; a : [no well its not a dream kyle]
q : [your mom told you]; a : [no mom i have to get you get that i have to get my mom before nobody knows to get me]
q 

### Export Checkpoints to Disk

In [65]:
# List checkpoints
#!ls
!rm seq2seq_model.ckpt-3000.meta
!ls

rm: cannot remove 'seq2seq_model.ckpt-3000.meta': No such file or directory
Bot_sp	Bot-sp	ckpt  datalab  idx_a.npy  idx_q.npy  metadata.pkl  raw_data


In [0]:
# Upload files to Google drive
import pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

# get the folder id where you want to save your file
ckpt_name = 'seq2seq_model.ckpt-7000'  

file = drive.CreateFile({'title': ckpt_name + '.meta'})
file.SetContentFile('./ckpt/' + ckpt_name + '.meta')
file.Upload() 
file = drive.CreateFile({'title': ckpt_name + '.index'})
file.SetContentFile('./ckpt/' + ckpt_name + '.index')
file.Upload() 
file = drive.CreateFile({'title': ckpt_name + '.data-00000-of-00001'})
file.SetContentFile('./ckpt/' + ckpt_name + '.data-00000-of-00001')
file.Upload() 

In [13]:
# Download files from Google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

folder_id = '1aBDUywmTyBqzQQRImYqTdZFb3M0UO8S2'

file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()

for i, file1 in enumerate(sorted(file_list, key = lambda x: x['title'])):
    print('Downloading {} from GDrive ({}/{})'.format(file1['title'], i+1, len(file_list)))
    file1.GetContentFile(file1['title'])

title: seq2seq_model.ckpt-4000.data-00000-of-00001, id: 1jqSOJWw2Oxm6CZKAbBrHSyFFXPBjujwn
title: seq2seq_model.ckpt-4000.index, id: 15kQuY0ARqkXl6JXrw66pylESaa0RpW8x
title: seq2seq_model.ckpt-4000.meta, id: 1BvhL6ALyaAlnLxsJW1OBzAs8-FzksKoq
title: checkpoint, id: 1ZUIprnuq2ev34eStCoClrYaQecrKVOlc
title: seq2seq_model.ckpt-13000.data-00000-of-00001, id: 1bLq5jM_3p01SmknrVGiwWlPmlzkGRtER
title: seq2seq_model.ckpt-13000.index, id: 128-UuvHvTnvrnjidXdr9ulamwl_275Nn
title: seq2seq_model.ckpt-13000.meta, id: 150GccZRcmmmkz0Sj9Htqv7-bPcm0NnJe
title: checkpoint, id: 18JV33lI7S0Dxq2_vpWe3PqGbxc-QU6c1
title: App lab, id: 1V-vJMQ_R71jhZ4kD6KW4W0OcG6ohriQIiRLKgdtEW18
title: seq2seq_model.ckpt-24000.data-00000-of-00001, id: 1R7EC-DMOeRjnE1mBc7V0yMmeY0TX3XyK
title: seq2seq_model.ckpt-24000.index, id: 1PKS13ylB5F73AvZTKfCK6qQ_CtxRNspN
title: seq2seq_model.ckpt-24000.meta, id: 1n5E_5prCQF8CCAvabXrs1tKCJHEeVN7K
title: checkpoint, id: 1WIcxd-L209CyB15Rd4hGT0h2ALUy4vaz
title: seq2seq_model.ckpt-21000.dat

In [14]:
# Move checkpoint files to ckpt directory

!ls
!rm -rf ckpt
!mkdir ckpt
!mv checkpoint ckpt/checkpoint
!mv seq2seq_model.ckpt-31000.data-00000-of-00001 ckpt/seq2seq_model.ckpt-31000.data-00000-of-00001
!mv seq2seq_model.ckpt-31000.meta ckpt/seq2seq_model.ckpt-31000.meta
!mv seq2seq_model.ckpt-31000.index ckpt/seq2seq_model.ckpt-31000.index
!ls ckpt

Bot_sp	    metadata.pkl
checkpoint  raw_data
datalab     seq2seq_model.ckpt-31000.data-00000-of-00001
idx_a.npy   seq2seq_model.ckpt-31000.index
idx_q.npy   seq2seq_model.ckpt-31000.meta
checkpoint				      seq2seq_model.ckpt-31000.index
seq2seq_model.ckpt-31000.data-00000-of-00001  seq2seq_model.ckpt-31000.meta


In [0]:
!kill -9 -1